# Table of Contents for 4.6 Combining & Exporting Data
    Importing libraries, set path, import files (orders_checked and orders_products_prior)
    Lesson: Check both files heads and shape
            Merge via outer both dataframes using an indicator
            Check head of new df
            Get value_counts for the indicator _merge
            Find missing values for the merged datafame
            Create dataset of nulls for product_id column, check head and shape
            Check shape of merged df
            Remove nulls from merged df
            Check the shape
            Look for duplicates: none
            Export as orders_products_combined.pkl
            Check shape again

# Importing Libraries

In [7]:
import pandas as pd
import numpy as np
import os

# Set path

In [8]:
path = r'C:\Users\valev\Desktop\0325 Instacart Basket Analysis'

# Import files

In [34]:
df_orders_checked = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'orders_checked.csv'), usecols=['order_id', 'user_id', 'order_number', 'orders_day_of_week', 'order_hour_of_day', 'days_since_prior_order'])

In [35]:

df_ords_prior = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'orders_products_prior.csv' ), index_col=None)

# Check Head, Dimensions


In [36]:
df_orders_checked.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order
0,2539329,1,1,2,8,NaN
1,2398795,1,2,3,7,15.0
2,473747,1,3,3,12,21.0
3,2254736,1,4,4,7,29.0
4,431534,1,5,4,15,28.0


In [37]:
df_ords_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [38]:
df_orders_checked.shape

(3421083, 6)

In [39]:
df_ords_prior.shape

(32434489, 4)

### Merge

In [44]:
df_merged_large = df_orders_checked.merge(df_ords_prior, on = 'order_id', how = 'outer', indicator = True)

In [45]:
df_merged_large.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge
0,1,112108,4,4,10,9.0,NaN,NaN,NaN,left_only
1,2,202279,3,5,9,8.0,33120.0,1.0,1.0,both
2,2,202279,3,5,9,8.0,28985.0,2.0,1.0,both
3,2,202279,3,5,9,8.0,9327.0,3.0,0.0,both
4,2,202279,3,5,9,8.0,45918.0,4.0,1.0,both


In [46]:
df_merged_large['_merge'].value_counts()

_merge
both          32434489
left_only       206209
right_only           0
Name: count, dtype: int64

# Check for null values

In [49]:
# Find Missing Values in df_merged_large
df_merged_large.isnull().sum()

order_id                        0
user_id                         0
order_number                    0
orders_day_of_week              0
order_hour_of_day               0
days_since_prior_order    2078068
product_id                 206209
add_to_cart_order          206209
reordered                  206209
_merge                          0
dtype: int64

In [51]:
# Create data set of nulls --- REMOVE ANY NULL IN PRODUCT_ID - There will be nothing to match but an empty order
df_nan = df_merged_large[df_merged_large['product_id'].isnull()==True]

In [52]:
df_nan.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge
0,1,112108,4,4,10,9.0,NaN,NaN,NaN,left_only
145,17,36855,5,6,15,1.0,NaN,NaN,NaN,left_only
322,34,35220,20,3,11,8.0,NaN,NaN,NaN,left_only
328,36,79431,23,6,18,30.0,NaN,NaN,NaN,left_only
332,38,42756,6,6,16,24.0,NaN,NaN,NaN,left_only


In [54]:
df_nan.shape

(206209, 10)

In [55]:
df_merged_large.shape

(32640698, 10)

In [58]:
#Create new df for only nonmissing records, excluding the 206,209 found above
df_merged_large_clean = df_merged_large[df_merged_large['product_id'].isnull() == False]

In [59]:
df_merged_large_clean.shape

(32434489, 10)

The number is now 206,209 records less

In [60]:
# Look for duplicates
df_dups = df_merged_large_clean[df_merged_large_clean.duplicated()]

In [63]:
df_dups

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge


There are no duplicates

# Export data to pkl

In [64]:
df_merged_large_clean.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_combined.pkl'))

In [65]:
df_merged_large_clean.shape

(32434489, 10)